In [20]:
# Imports
import pandas as pd
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from config import database_url

In [21]:
# Look at the databse_url

In [22]:
# Set up the connection to our database, and configure our jupyter notebook to query the DB

# Set up the connection and environment to our database
engine = create_engine(database_url)
connection = engine.connect()

# Get information about the tables and have reference to them
Base = automap_base()

# Get the tables
Base.prepare(engine, reflect = True)

# Find what classes (tables) exist
print(Base.classes.keys())

# inspector = inspect(engine)
# inspector.get_table_names()

# Save references to each table (class)
Dept_Manager = Base.classes.dept_manager
Departments = Base.classes.departments
Employees = Base.classes.employees
Dept_Employee = Base.classes.dept_employee
Titles = Base.classes.titles
Salaries = Base.classes.salaries

['employees', 'dept_employee', 'departments', 'dept_manager', 'salaries', 'titles']


In [23]:
# Create a link to the database and execute queries
session = Session(engine)

In [24]:
from sqlalchemy import inspect

In [25]:
# Task: Bar chart -> average salary by title
# What table(s) do we need to use -> titles, salaries, employees

# RAW SQL Version:
# SELECT title, AVG(salary)
# FROM employees
# INNER JOIN titles
# ON (employees.emp_no = titles.emp_no)
# INNER JOIN salaries
# ON (employees.emp_no = salaries.emp_no)
# GROUP BY title

# SQLAlchemy Version:
# db.session.query(User.email).\
#     join(Account, Account.organization == User.organization).\
#     filter(Account.name == 'some name')

# Join all 3 tables
query_result = session.query(Employees.emp_no, Titles.title, Salaries.salary)\
    .join(Titles, Employees.emp_no == Titles.emp_no)\
    .join(Salaries, Employees.emp_no == Salaries.emp_no)

query_result

In [26]:
query_result.all()

[(10005, 'Senior Staff', 78228),
 (10005, 'Staff', 78228),
 (10010, 'Engineer', 72488),
 (10011, 'Staff', 42365),
 (10018, 'Engineer', 55881),
 (10018, 'Senior Engineer', 55881),
 (10035, 'Engineer', 41538),
 (10035, 'Senior Engineer', 41538),
 (10043, 'Engineer', 49324),
 (10043, 'Senior Engineer', 49324),
 (10045, 'Engineer', 41971),
 (10058, 'Senior Staff', 52787),
 (10078, 'Engineer', 47280),
 (10081, 'Engineer', 55786),
 (10081, 'Senior Engineer', 55786),
 (10085, 'Senior Engineer', 40000),
 (10096, 'Engineer', 61395),
 (10104, 'Senior Staff', 45312),
 (10126, 'Senior Staff', 40000),
 (10126, 'Staff', 40000),
 (10167, 'Engineer', 56990),
 (10167, 'Senior Engineer', 56990),
 (10176, 'Senior Staff', 54635),
 (10176, 'Staff', 54635),
 (10178, 'Staff', 50568),
 (10181, 'Senior Engineer', 40000),
 (10185, 'Senior Staff', 68334),
 (10185, 'Staff', 68334),
 (10228, 'Senior Staff', 79474),
 (10228, 'Staff', 79474),
 (10231, 'Staff', 40000),
 (10232, 'Senior Staff', 89606),
 (10232, 'Staff

In [27]:
func.avg()

<sqlalchemy.sql.functions.Function at 0x202bca62160; avg>

In [28]:
# Query the average salary for each title
salaries_by_title = session.query(Titles.title, func.avg(Salaries.salary))\
    .join(Titles, Employees.emp_no == Titles.emp_no)\
    .join(Salaries, Employees.emp_no == Salaries.emp_no)\
    .group_by(Titles.title)
    
salaries_by_title.all()

InvalidRequestError: Can't join table/selectable 'titles' to itself